Exploring the billboarddata.csv data set

Some of the times dont make sense (use different convention)? Some artists are numbered.

Dates are not in standard date-time format.

regex on index line
u
:'<,'>s/,\(\d\)/,x\1/g
:'<,'>s/\s/_/g

In [ ]:
import pandas as pd
import numpy as np

import datetime as dt
import time

billboard = pd.DataFrame.from_csv('BillboardData.csv',index_col=False)
billboard.tail()

In [3]:
billboard.iloc[265][:40]

/Users/swahl/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1159: DtypeWarning: Columns (32,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,year,yearly_rank,source,prefix,ch,x40,x10,pk,high,verified,...,x82nd_week,x83rd_week,x84th_week,x85th_week,x86th_week,x87th_week,x88th_week,x89th_week,x90th_week,x91st_week
40129,2014,405,h,2014-405,20,18,5,1,8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40130,2014,406,h,2014-406,20,18,8,1,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40131,2014,407,h,2014-407,1,0,0,1,45,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40132,2014,408,h,2014-408,1,0,0,1,50,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40133,2014,409,h,2014-409,1,0,0,1,51,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
cols = billboard.keys()
cols

Index([u'year', u'yearly_rank', u'source', u'prefix', u'ch', u'x40', u'x10', u'pk', u'high', u'verified', u'artist', u'artist_standardized', u'artist_inverted', u'featured', u'unfeatured', u'album', u'b-side', u'track', u'time', u'time_source', u'explicit', u'time_(album)', u'artist_id', u'symbl', u'reissue', u'label/number', u'media', u'stereo_(55-68)', u'pic_sleeve', u'genre', u'comments', u'written_by', u'temp_1', u'date_entered', u'date_peaked', u'x1st_week', u'x2nd_week', u'x3rd_week', u'x4th_week', u'x5th_week', u'x6th_week', u'x7th_week', u'x8th_week', u'x9th_week', u'x10th_week', u'x11th_week', u'x12th_week', u'x13th_week', u'x14th_week', u'x15th_week', u'x16th_week', u'x17th_week', u'x18th_week', u'x19th_week', u'x20th_week', u'x21st_week', u'x22nd_week', u'x23rd_week', u'x24th_week', u'x25th_week', u'x26th_week', u'x27th_week', u'x28th_week', u'x29th_week', u'x30th_week', u'x31st_week', u'x32nd_week', u'x33rd_week', u'x34th_week', u'x35th_week', u'x36th_week', u'x37th_week', 

In [7]:
import datetime as dt
def convertDate(s):
    fmt = '%m/%d/%y'
    #fmt = '%Y-%m-%d'

    try:
        return dt.datetime.strptime(s, fmt)
    except:
        return None
    
def timeInSec(t):
    fmt = '%M:%S'
    try:
        x = time.strptime(t,fmt)
        return dt.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds()
    except:
        return None

fmt = '%Y-%m-%d'
dt.datetime.strptime('2014-01-29',fmt)

datetime.datetime(2014, 1, 29, 0, 0)

In [7]:
# clean non-readable stuff
for col in cols:
    if billboard[col].dtype == np.dtype('O'):
        billboard[col] = billboard[col].str.decode('iso-8859-1').str.encode('utf-8')

In [8]:
# convert time to seconds (NaN if invalid format)
#billboard.time.apply(timeInSec)
billboard.time = billboard.time.apply(timeInSec)
billboard['time_(album)'] = billboard['time_(album)'].apply(timeInSec)
billboard.time

0     253
1     211
2     199
3     200
4     221
5     291
6     233
7     265
8     255
9     243
10    255
11    246
12    260
13    257
14    205
...
40119   NaN
40120   NaN
40121   NaN
40122   NaN
40123   NaN
40124   NaN
40125   NaN
40126   NaN
40127   NaN
40128   NaN
40129   NaN
40130   NaN
40131   NaN
40132   NaN
40133   NaN
Name: time, Length: 40134, dtype: float64

In [9]:
# convert to integers
weeks = [s for s in cols if 'week' in s]
for field in ['ch','x40','x10','high','temp_1'] + weeks:
    billboard[field] = billboard[field].convert_objects(convert_numeric=True)

billboard.x7th_week

0     57
1     55
2    NaN
3     56
4     71
5    NaN
6     15
7     60
8     40
9     17
10    19
11    26
12    39
13    25
14    89
...
40119   NaN
40120   NaN
40121    14
40122    20
40123   NaN
40124   NaN
40125   NaN
40126   NaN
40127   NaN
40128   NaN
40129   NaN
40130   NaN
40131   NaN
40132   NaN
40133   NaN
Name: x7th_week, Length: 40134, dtype: float64

In [10]:
# convert dates (rows with non-usable dates appear to be duplicates so remove them)
billboard.date_entered = billboard.date_entered.apply(convertDate)
billboard.date_peaked = billboard.date_peaked.apply(convertDate)
billboard = billboard[(pd.notnull(billboard.date_entered)) & (pd.notnull(billboard.date_peaked))]
billboard = billboard.sort('prefix')
billboard[['prefix','year','date_entered','date_peaked','artist','track','genre','album','time']]

,prefix,year,date_entered,date_peaked,artist,track,genre,album,time
4670,1944_001,1944,2044-05-13,2044-08-05,Bing Crosby,Swinging On A Star,Vocal,NaN,149
1461,1944_002,1944,2044-01-01,2044-01-15,Andrews Sisters,Shoo-Shoo Baby,Vocal,NaN,170
4901,1944_003,1944,2044-11-25,2044-12-16,Bing Crosby & Andrews Sisters,Don't Fence Me In,Vocal,NaN,183
20765,1944_004,1944,2044-01-15,2044-03-04,Jimmy Dorsey & His Orchestra,Besame Mucho,Jazz,NaN,180
17553,1944_005,1944,2044-04-15,2044-06-10,Harry James & His Orchestra (Vocal Dick Haymes),I'll Get By (As Long As I Have You),Jazz,NaN,162
4902,1944_006,1944,2044-09-23,2044-10-14,Bing Crosby & Andrews Sisters,A Hot Time in the Town of Berlin,Vocal,NaN,171
27191,1944_007,1944,2044-06-03,2044-10-07,Mills Brothers,You Always Hurt The One You Love,Vocal,NaN,202
4671,1944_008,1944,2044-03-11,2044-04-29,Bing Crosby,San Fernando Valley,Vocal,NaN,191
4672,1944_010,1944,2044-04-15,2044-05-06,Bing Crosby,I Love You,Vocal,NaN,155
26763,1944_011,1944,2044-02-26,2044-03-18,Merry Macs,Mairzy Doats,NaN,NaN,162


In [32]:
post90 = billboard[billboard.year > 1989]
post00 = billboard[billboard.year > 1999]
post05 = billboard[billboard.year > 2004]

In [33]:
billboard.to_csv('BillboardData_mod.csv',index_label='id',encoding="utf-8")
post90.to_csv('BillboardData_mod_1990.csv',index_label='id',encoding="utf-8")
post00.to_csv('BillboardData_mod_2000.csv',index_label='id',encoding="utf-8")
post05.to_csv('BillboardData_mod_2005.csv',index_label='id',encoding="utf-8")